# 扩展说明：指令微调Llama 2

这篇博客是一篇来自Meta AI，关于指令微调Llama 2的扩展说明。旨在聚焦构建指令数据集，有了它，我们则可以使用自己的指令来微调Llama 2基础模型。

目标是构建一个能够基于输入内容来生成指令的模型。这么做背后的逻辑是，模型如此就可以由其他人生成自己的指令数据集。这在当想开发私人个性化定制模型，如发送推特、写邮件等，时很方便。这也意味着你可以通过你的邮件来生成一个指令数据集，然后用它来训练一个模型来为你写邮件。

好，那我们来开始吧？我们将进行：

1. 定义应用场景细节并创建指令的提示词模板
2. 构建指令数据集
3. 使用 `trl` 与 `SFTTrainer` 指令微调Llama 2
4. 测试模型、进行推理

## 1. 定义应用场景细节并创建指令的提示词模板

在描述应用场景前，我们要更好的理解一下究竟什么是指令。

>指令是一段文本或提供给大语言模型，类似Llama，GPT-4或Claude，使用的提示词，用来指导它去生成回复。指令可以让人们做到把控对话，约束模型输出更自然、实用的输出，并使这些结果能够对齐用户的目的。制作清晰的、整洁的指令则是生成高质量对话的关键。
>

指令的例子如下表所示。

| 能力 | 示例指令 |
| --- | --- |
| 头脑风暴 | 提供一系列新口味的冰淇淋的创意。 |
| 分类 | 根据剧情概要，将这些电影归类为喜剧、戏剧或恐怖片。 |
| 确定性问答 | 用一个单词回答“法国的首都是哪里？” |
| 生成 | 用罗伯特·弗罗斯特的风格写一首关于大自然和季节变化的诗。 |
| 信息提取 | 从这篇短文中提取主要人物的名字。 |
| 开放性问答 | 为什么树叶在秋天会变色？用科学的理由解释一下。 |
| 摘要 | 用2-3句话概括一下这篇关于可再生能源最新进展的文章。 |

如开头所述，我们想要微调模型，以便根据输入（或输出）生成指令。 我们希望将其用作创建合成数据集的方法，以赋予LLM和代理个性化能力。

把这个想法转换成一个基础的提示模板，按照[Alpaca格式](https://github.com/tatsu-lab/stanford_alpaca#data-release).


```python
### 指令:
使用下面的输入创建一条指令，该指令可以使用LLM生成输入文本。

### 输入文本:
亲爱的 [领导名字]，
此封信件是想申请我下周一到周四的带薪假期。

下周我有一些私人事务要处理，不得不离开办公
室。我想尽提前地通知您，这样您就可以在我离
开期间做好相应的安排。

如果您需要任何其他信息，或者对我下周休假有
任何疑问，请联系我。谢谢您对此的受理。

谢谢，[你的名字]

### 回复:
给我的领导写封邮件，说我下周8月1日到8月4日需要请假。
```

## 2. 创建指令数据集

在定义了我们的应用场景和提示模板后，我们需要创建自己的指令数据集。创建高质量的指令数据集是获得良好模型性能的关键。研究表明，[“对齐，越少越好”](https://arxiv.org/abs/2305.11206)表明，创建高质量、低数量(大约1000个样本)的数据集可以达到与低质量、高数量的数据集相同的性能。

创建指令数据集有几种方法，包括:

1. 使用现有数据集并将其转换为指令数据集，例如[FLAN](https://huggingface.co/datasets/SirNeural/flan_v2)
2. 使用现有的LLM创建合成指令数据集，例如[Alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca)
3. 人力创建指令数据集，例如[Dolly](https://huggingface.co/datasets/databricks/databricks-dolly-15k)。

每种方法都有其优缺点，这取决于预算、时间和质量要求。例如，使用现有数据集是最简单的，但可能不适合您的特定用例，而使用人力可能是最准确的，但必然耗时、昂贵。也可以结合几种不同方法来创建指令数据集，如[Orca: Progressive Learning from Complex Explanation Traces of GPT-4.](https://arxiv.org/abs/2306.02707)。

为了简单起见，我们将使用 **[Dolly](https://huggingface.co/datasets/databricks/databricks-dolly-15k)** ，这是一个开源的指令跟踪记录数据集，由数千名Databricks员工在 **[InstructGPT paper](https://arxiv.org/abs/2203.02155)** 中描述的几个行为类别中生成，包括头脑风暴、分类、确定性回答、生成、信息提取、开放性回答和摘要。

开始编程吧，首先，我们来安装依赖项。

In [ ]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

加载 **`databricks/databricks-dolly-15k`** 数据集，我们使用 🤗 Datasets library的 **`load_dataset()`** 方法。

In [1]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])
# dataset size: 15011

Found cached dataset json (/home/ubuntu/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


dataset size: 15011
{'instruction': 'On what month and day was Antwan Deon Odom born?', 'context': 'Antwan Deon Odom (born September 24, 1981) is a former American football defensive end. He was drafted by the Tennessee Titans in the second round of the 2004 NFL Draft. He played college football at Alabama. He has also played for the Cincinnati Bengals.', 'response': 'September 24', 'category': 'closed_qa'}


为了指导我们的模型，我们需要将我们的结构化示例转换为通过指令描述的任务集合。我们定义一个 **`formatting_function`** ，它接受一个样本并返回一个符合格式指令的字符串。

In [2]:
def format_instruction(sample):
	return f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 

### Input:
{sample['response']}

### Response:
{sample['instruction']}
"""

Let's test our formatting function on a random example.

In [3]:
from random import randrange

print(format_instruction(dataset[randrange(len(dataset))]))

### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 

### Input:
U.S. cities in the list: Chicago, New York, Atlanta, Honolulu, Austin
U.S. states in the list: Montana, New York, Florida, South Dakota

### Response:
Which are the U.S. cities and which are U.S. states in this list? Montana, Chicago, New York, Atlanta, Honolulu, Florida, Austin, South Dakota



## 3. Instruction-tune Llama 2 using `trl` and the `SFTTrainer`

 We will use the recently introduced method in the paper "[QLoRA: Quantization-aware Low-Rank Adapter Tuning for Language Generation](https://arxiv.org/abs/2305.14314)" by Tim Dettmers et al. QLoRA is a new technique to reduce the memory footprint of large language models during finetuning, without sacrificing performance. The TL;DR; of how QLoRA works is:

- Quantize the pre-trained model to 4 bits and freeze it.
- Attach small, trainable adapter layers. (LoRA)
- Finetune only the adapter layers while using the frozen quantized model for context.

If you want to learn more about QLoRA and how it works, I recommend you to read the [Making LLMs even more accessible with bitsandbytes, 4-bit quantization and QLoRA](https://huggingface.co/blog/4bit-transformers-bitsandbytes) blog post.

### Flash Attention

Flash Attention is a an method that reorders the attention computation and leverages classical techniques (tiling, recomputation) to significantly speed it up and reduce memory usage from quadratic to linear in sequence length. It is based on the paper "[FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness](https://arxiv.org/abs/2205.14135)".
The TL;DR; accelerates training up to 3x. Learn more at [FlashAttention](https://github.com/Dao-AILab/flash-attention/tree/main). Flash Attention is currently only available for Ampere (A10, A40, A100, ...) & Hopper (H100, ...) GPUs. You can check if your GPU is supported and install it using the following command:

_Note: If your machine has less than 96GB of RAM and lots of CPU cores, reduce the number of `MAX_JOBS`. On the `g5.2xlarge` we used `4`._

```bash
python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'"
pip install ninja packaging
MAX_JOBS=4 pip install flash-attn --no-build-isolation
```

_Installing flash attention can take quite a bit of time (10-45 minutes)._

The example supports the use of Flash Attention for all Llama checkpoints, but is not enabled by default. To use Flash Attention comment in the code block below wich says  `# COMMENT IN TO USE FLASH ATTENTION`.


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = False

# COMMENT IN TO USE FLASH ATTENTION
# replace attention with flash attention 
# if torch.cuda.get_device_capability()[0] >= 8:
#     from utils.llama_patch import replace_attn_with_flash_attn
#     print("Using flash attention")
#     replace_attn_with_flash_attn()
#     use_flash_attention = True


# Hugging Face model id
model_id = "NousResearch/Llama-2-7b-hf" # non-gated
# model_id = "meta-llama/Llama-2-7b-hf" # gated


# BitsAndBytesConfig int-4 config 
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache=False, device_map="auto")
model.config.pretraining_tp = 1 

# Validate that the model is using flash attention, by comparing doc strings
if use_flash_attention:
    from utils.llama_patch import forward    
    assert model.model.layers[0].self_attn.forward.__doc__ == forward.__doc__, "Model is not using flash attention"


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `SFTTrainer`  supports a native integration with `peft`, which makes it super easy to efficiently instruction tune LLMs. We only need to create our `LoRAConfig` and provide it to the trainer.

In [5]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM", 
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

Before we can start our training we need to define the hyperparameters (`TrainingArguments`) we want to use.

In [6]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="llama-7-int4-dolly",
    num_train_epochs=3,
    per_device_train_batch_size=6 if use_flash_attention else 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True # disable tqdm since with packing values are in correct
)

We now have every building block we need to create our `SFTTrainer` to start then training our model.

In [7]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction, 
    args=args,
)

Start training our model by calling the `train()` method on our `Trainer` instance.

In [8]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


The training without Flash Attention enabled took 03:08:00 on a `g5.2xlarge`. The instance costs `1,212$/h` which brings us to a total cost of `3.7$`. 
The training with Flash Attention enabled took 02:08:00 on a `g5.2xlarge`. The instance costs `1,212$/h` which brings us to a total cost of `2.6$`.

The results using Flash Attention are mind blowing and impressive, 1.5x faster and 30% cheaper.

## 4. Test Model and run Inference

After the training is done we want to run and test our model. We will use `peft` and `transformers` to load our LoRA adapter into our model.

In [ ]:
if use_flash_attention:
    # unpatch flash attention
    from utils.llama_patch import unplace_flash_attn_with_attn
    unplace_flash_attn_with_attn()
    
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer


args.output_dir = "llama-7-int4-dolly"

# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
) 
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)

Let’s load the dataset again with a random sample to try to generate an instruction.

In [ ]:
from datasets import load_dataset 
from random import randrange


# Load dataset from the hub and get a sample
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
sample = dataset[randrange(len(dataset))]

prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 

### Input:
{sample['response']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)

print(f"Prompt:\n{sample['response']}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample['instruction']}")

Nice! our model works! If want to accelerate our model we can deploy it with [Text Generation Inference](https://github.com/huggingface/text-generation-inference). Therefore we would need to merge our adapter weights into the base model.

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
) 

# Merge LoRA and base model
merged_model = model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")

# push merged model to the hub
# merged_model.push_to_hub("user/repo")
# tokenizer.push_to_hub("user/repo")